In [2]:
%matplotlib inline
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=2

import os
import cv2
import numpy as np
import torch
from torch.autograd import Variable
import quat_math
import pickle
import glob

from PIL import Image
import scipy.io as scio
from functools import partial
from object_pose_utils.utils import to_np, to_var
from object_pose_utils.utils.display import *

import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = 20, 12
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import

import warnings
warnings.filterwarnings('ignore')

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=2


## Dataset Information

In [2]:
from object_pose_utils.datasets.ycb_dataset import YcbDataset as YCBDataset
from object_pose_utils.datasets.pose_dataset import OutputTypes as otypes

dataset_root = '/ssd0/datasets/ycb/YCB_Video_Dataset'
object_list = list(range(1,22))
mode = "test"

output_format = [otypes.OBJECT_LABEL,
                 otypes.QUATERNION, 
                 otypes.TRANSLATION, 
                 otypes.TRANSFORM_MATRIX,
                ]

dataset = YCBDataset(dataset_root, mode=mode,
                     object_list = list(range(1,22)),
                     output_data = output_format,
                     resample_on_error = False,
                     add_syn_background = False,
                     add_syn_noise = False,
                     use_posecnn_data = True,
                     #preprocessors = [InplaneRotator()],
                     #postprocessors = [ImageNormalizer()],
                     image_size = [640, 480], num_points=1000)

feature_root = '/scratch/bokorn/results/dense_fusion_local_orig_feat/'
grid_vertices = torch.load(os.path.join(feature_root, 'grid',
    '{}_vertices.pt'.format(dataset.classes[1])))

In [11]:
from se3_distributions.losses.bingham_loss import isoLikelihood, duelLikelihood, mLikelihood

In [12]:
results_bing = 'ral_results/single_img_bing_iso_bing_duel_results.npz'

bing_data = np.load(results_bing, allow_pickle=True)
bing_lik_distribution = bing_data['lik_distribution'].item()['bing_duel']

In [13]:
from tqdm import tqdm_notebook as tqdm

bing_parameters = {}

with torch.no_grad():
    for j, data in enumerate(tqdm(dataset)):
        obj, quat, trans, mat = data
        path = dataset.getPath(j)
        vid_id, img_id = path.split('/')
        obj_id = obj.item()
        
        if(j not in bing_lik_distribution[obj_id].keys()):
            continue
        bing_dist = bing_lik_distribution[obj_id][j]
        
        if(obj_id not in bing_parameters.keys()):
            bing_parameters[obj_id] = {}
        if(vid_id not in bing_parameters[obj_id].keys()):
            bing_parameters[obj_id][vid_id] = {}

        bing_parameters[obj_id][vid_id][img_id] = {'sig':to_np(bing_dist.sig).item(), 
                                                   'mean':to_np(bing_dist.mean_q)}
        

NameError: name 'dataset' is not defined

In [16]:
results_hist = 'ral_results/pcnn_single_img_hist_comp_fc6_results.npz'

hist_data = np.load(results_hist, allow_pickle=True)
hist_lik_distribution = hist_data['lik_distribution'].item()['hist_comp_fc6']

In [19]:
from tqdm import tqdm_notebook as tqdm

hist_parameters = {}

with torch.no_grad():
    for j, data in enumerate(tqdm(dataset)):
        obj, quat, trans, mat = data
        path = dataset.getPath(j)
        vid_id, img_id = path.split('/')
        obj_id = obj.item()
        
        if(j not in hist_lik_distribution[obj_id].keys()):
            continue
        bin_vals = hist_lik_distribution[obj_id][j]
        
        if(obj_id not in hist_parameters.keys()):
            hist_parameters[obj_id] = {}
        if(vid_id not in hist_parameters[obj_id].keys()):
            hist_parameters[obj_id][vid_id] = {}
        hist_parameters[obj_id][vid_id][img_id] = {'bin_vals':bin_vals.flatten()}
        
        

In [3]:
#np.savez('tracking_distributions.npz', 
#         bing_parameters=bing_parameters, 
#         hist_parameters=hist_parameters)

NameError: name 'bing_parameters' is not defined

In [4]:
tracking_distributions = np.load('tracking_distributions.npz', allow_pickle=True)

In [5]:
bing_parameters = tracking_distributions['bing_parameters'].item()
hist_parameters = tracking_distributions['hist_parameters'].item()

In [7]:
hist_parameters[1]['0048']['000001']
bing_parameters[1]['0048']['000001']


{'bin_vals': array([7.5199401e-13, 8.3389767e-07, 1.5088676e-12, ..., 6.3490960e-15,
        9.6765661e-14, 5.8215780e-16], dtype=float32)}

In [8]:
from object_pose_utils.datasets.ycb_dataset import YcbDataset as YCBDataset
from object_pose_utils.datasets.image_processing import ImageNormalizer
from object_pose_utils.datasets.inplane_rotation_augmentation import InplaneRotator

from object_pose_utils.datasets.pose_dataset import OutputTypes as otypes

dataset_root = '/ssd0/datasets/ycb/YCB_Video_Dataset'
object_list = list(range(1,22))
mode = "test"

output_format = [otypes.OBJECT_LABEL,
                 otypes.QUATERNION, 
                 otypes.TRANSLATION, 
                 otypes.TRANSFORM_MATRIX,
                ]

dataset = YCBDataset(dataset_root, mode=mode,
                     object_list = list(range(1,22)),
                     output_data = output_format,
                     resample_on_error = False,
                     add_syn_background = False,
                     add_syn_noise = False,
                     use_posecnn_data = True,
                     #preprocessors = [InplaneRotator()],
                     #postprocessors = [ImageNormalizer()],
                     image_size = [640, 480], num_points=1000)

In [12]:
obj, quat, trans, mat = dataset[0]

vid_id, frame_id = dataset.getPath(0).split('/')
print(hist_parameters[obj.item()][vid_id][frame_id])
print(bing_parameters[obj.item()][vid_id][frame_id])

{'bin_vals': array([7.5199401e-13, 8.3389767e-07, 1.5088676e-12, ..., 6.3490960e-15,
       9.6765661e-14, 5.8215780e-16], dtype=float32)}
{'sig': 144.1236114501953, 'mean': array([-0.74177265,  0.279474  , -0.2430372 , -0.5591071 ], dtype=float32)}


In [9]:
scio.savemat('tracking_distributions.mat', {'bing_parameters':bing_parameters, 'hist_parameters':hist_parameters})